In [569]:
#This notebook aims to measure the SNR of each star we are looking at in K-band

import numpy as np
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, ListedColormap
from scipy.stats import sigmaclip, binned_statistic as bs
from scipy.spatial import KDTree
from astropy.stats import sigma_clip
import pandas as pd
from multiprocessing import Pool
import time
import math
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
import astropy.time
from astropy.stats import sigma_clip
%matplotlib inline

In [570]:
import jdaviz

In [571]:
from jdaviz import SpecViz
from jdaviz import CubeViz
from jdaviz import MosViz
from jdaviz import Imviz

In [572]:
import warnings
warnings.simplefilter('ignore')

In [573]:
#https://spacetelescope.github.io/notebooks/notebooks/DrizzlePac/align_to_catalogs/align_to_catalogs.html
import astropy.units as u
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import astroquery

from astropy.io import fits
from astropy.table import Table
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
from astroquery import sha
from astroquery.gaia import Gaia
from astroquery.mast import Mast
from astroquery.sdss import SDSS

#from ccdproc import ImageFileCollection
from IPython.display import Image

from drizzlepac import tweakreg
from drizzlepac import astrodrizzle

In [574]:
# Login
# https://auth.mast.stsci.edu/token
Observations.login(token="996ce91baedf446987de44b80d454dc9")
sessioninfo = Observations.session_info()

INFO:astroquery:MAST API token accepted, welcome Ori Fox
2021-06-11 09:43:42,523 - stpipe - INFO - MAST API token accepted, welcome Ori Fox
INFO:stpipe:MAST API token accepted, welcome Ori Fox


INFO: MAST API token accepted, welcome Ori Fox [astroquery.mast.auth]
eppn: ofox@stsci.edu
ezid: ofox
attrib.uuid: c8e19103-c1b6-4787-91ec-c35d0f4ea5b2
attrib.first_name: Ori
attrib.last_name: Fox
attrib.display_name: Ori Fox
attrib.internal: 1
attrib.department: MIRI
attrib.email: ofox@stsci.edu
attrib.Jwstcalengdataaccess: true
anon: False
scopes: ['mast:user:info', 'mast:exclusive_access']
session: None
token: 996ce91b...


# Dick Shaw's Functions

In [620]:
SERVER = 'mast'

RET_COLUMNS = ['proposal_id','dataURL','obsid','obs_id']
#SERVICES = {
#            'SI_search': 'Mast.Jwst.Filtered.Miri',
#            'Caom_search':'Mast.Caom.Filtered.JwstOps',
#            'Product_search':'Mast.Caom.Products.JwstOps'
#            }
SERVICES = {
            'SI_search': 'Mast.Jwst.Filtered.Miri',
            'Caom_search':'Mast.Caom.Filtered.JwstTest2',
            'Product_search':'Mast.Caom.Products.JwstTest2'
            }

In [616]:
def set_server_endpoints(server_name):
    '''
    Set server endpoints for AstroQuery.
    
    Parameters
    ----------
    server_name : str
        Name of REST-ful server corresponding to JWST cluster.
    '''
    server = 'https://{}.stsci.edu'.format(server_name)
    JwstObs = Mast()
    JwstObs._portal_api_connection.MAST_REQUEST_URL = server + "/portal_jwst/Mashup/Mashup.asmx/invoke"
    JwstObs._portal_api_connection.MAST_DOWNLOAD_URL = server + "/jwst/api/v0.1/download/file"
    JwstObs._portal_api_connection.COLUMNS_CONFIG_URL = server + "/portal_jwst/Mashup/Mashup.asmx/columnsconfig"
    JwstObs._portal_api_connection.MAST_BUNDLE_URL = server + "/jwst/api/v0.1/download/bundle"
    return JwstObs

In [617]:
def observation_query(JwstObs, progID, instrument, mjd_min, filt):
    '''
    Perform query for observations matching JWST instrument and program ID
    that began after a particular date. 
    
    Parameters
    ----------
    JwstObs : object
        astroquery observation object
    progID
        JWST program identifier
    instrument
        JWST instrument used to obtain the observations
    mjd_min
        Date (MDJ) after which observations will be selected. This is the 
        lower end of the parameter range; the upper end is chosen to be about 
        10 years hence. 
    '''
    columns = ','.join(RET_COLUMNS)
    #service = SERVICES['Caom_search']
    service = SERVICES['Caom_search']
    params = {"columns":columns,
              "filters":[
                  {"paramName":"obs_collection","values":["JWST"]},
                  {"paramName":"instrument_name","values":[instrument]},
                  {"paramName":"proposal_id","values":[progID]},
                  {"paramName":"filters","values":[filt]},
                  {"paramName":"t_min",
                    "values":[{"min":mjd_min-60.,"max":mjd_min+60.}]},
              ]}
    obsTable = JwstObs.service_request(service, params)
    return obsTable

In [552]:
def product_query(JwstObs, obsTable):
    '''
    Perform query for data products based on obs_id's in observation table
    
    Parameters
    ----------
    JwstObs : object
        astroquery observation object
    obsTable : astropy table object
    '''
    obs_ids = ','.join(obsTable['obsid'])
    service = SERVICES['Product_search']
    params = {"obsid":obs_ids,
              "format":"json"
              }
    productTable = JwstObs.service_request(service, params)
    return productTable
    

In [553]:
def product_filter(table, prodList, gs_omit=True):
    '''
    Filter the list of products based on product semantic type
    
    Parameters
    ----------
    table : object
        astropy table of products associated with observations
    prodList : list of str
        list of product type extensions to select
    gs_omit
        Omit associated guide-star products?
    '''
    mask = np.full(len(table), False)
    gs_text = '_gs-'
    for r in table:
        mk = False
        fileName = r['productFilename']
        for p in prodList:
            if p in fileName:
                mk = mk | True
                if gs_text in fileName:
                    mk = not gs_omit
    
        mask[r.index] = mk
    
    return mask



In [554]:
def fetch_products(JwstObs, product_table):
    '''
    Retrieve specified data products from the MAST web service
    
    Parameters
    ----------
    JwstObs : object
        astroquery observation object
    product_table : object
        astropy table of selected data products
    '''
    uri_list_str = '&'.join(["uri=" + x for x in product_table["dataURI"]])
    now_str = ''.join(str(x).zfill(2) for x in Time.now().ymdhms).split('.')[0]
    local_path = 'mastDownload_' + now_str + '.sh'
    download_url = JwstObs._portal_api_connection.MAST_BUNDLE_URL + ".sh?" + uri_list_str
    JwstObs._download_file(download_url, local_path)
    
    return local_path



In [555]:
def fetch_files(progID, instrument, start_date, filt, extension, gs_omit, download):
    '''
    Driver to query for and retriev data files. The steps are: 
    1. Set the configuration for the web service calls
    2. Query for MAST observations 
    3. Query for data products associated with the observations
    4. Filter the list of products for the deisred types (e.g., "_uncal.fits")
    5. Report and/or retieve the data products
    
    Parameters
    ----------
    prodID : str
        ID of JWST program for which data were obtained
    instrument : str
        Name of JWST instrument used to obtain data
    start_date : float
        MJD date after which observations will be searched
    gs_omit : bool
        Omit associated guide-star data files?
    download : bool
        Generate curl script to download selected data files?
    '''
    JwstObs = set_server_endpoints(SERVER)
    obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
    if len(obsTable) > 0:
        products = product_query(JwstObs, obsTable)
        mask = product_filter(products, [extension], gs_omit)
        selected_products = products[mask]
        selected_products['productFilename'].pprint(max_lines=-1, show_name=False)
        if (download):
            path = fetch_products(JwstObs, selected_products)
    else:
        print("No matching observations")
        path = 'Null'
    
    return path

# Older Code

https://outerspace.stsci.edu/pages/viewpage.action?spaceKey=JARI&title=JWST+Archive%3A+Retrieve+Observations\
https://github.com/spacetelescope/JWST_Planned_Observations/blob/master/Querying_JWST_in_astroquery.ipynb\
https://mast.stsci.edu/api/v0/_c_a_o_mfields.html\

In [78]:
#APT_LINK = 'http://www.stsci.edu/cgi-bin/get-proposal-info?id={}&observatory=JWST'
#
#def get_program_URL(program_id):
#    """
#    Generate the URL for program status information, given a program ID. 
#    """
#    return APT_LINK.format(program_id)

In [79]:
#import astropy
#from astropy import table
#from astropy.table import Table
#from astropy import units as u
#from astropy.coordinates import Angle
#
#def display_results(obs):
#    """
#    Simple display of results related to identifying potentially duplicating targets.
#    Observation program title is truncated for presentation in this notebook
#    """
#    # build the URL to the JWST programs.
#    obs['proposal_URL'] = [get_program_URL(x) for x in obs['proposal_id']]
#    obs['obs_title'] = [x[:70] for x in obs['obs_title']]
#    obs['obs_title'].info.format = '<'
#    obs['target_name', 'instrument_name', 'filters', 'dataproduct_type', 't_exptime', 
#        'proposal_id'].pprint(max_lines=40, max_width=90)
#    
#    print("\nUnique Program Titles:")
#    table.unique(obs, keys=['proposal_id'])['proposal_id','obs_title'].pprint(max_width=100)
#    print("\nUnique URLs to status of existing programs:")
#    for i in set(obs['proposal_URL']):
#        print(i)

In [80]:
#server = "https://mast.stsci.edu"
#JwstObs = Mast()
#JwstObs._portal_api_connection.MAST_REQUEST_URL = server + "/portal_jwst/Mashup/Mashup.asmx/invoke"
#JwstObs._portal_api_connection.MAST_DOWNLOAD_URL = server + "/jwst/api/v0.1/download/file"
#JwstObs._portal_api_connection.COLUMNS_CONFIG_URL = server + "/portal_jwst/Mashup/Mashup.asmx/columnsconfig"
#JwstObs._portal_api_connection.MAST_BUNDLE_URL = server + "/jwst/api/v0.1/download/bundle"

In [84]:
#instrument = 'Nirspec'
#program = '00675'
#FIELDS = [
#        'program',
#        'observtn',
#        'instrume',
#        'productLevel',
#        ]

In [124]:
#service = 'Mast.Jwst.Filtered.' + instrument
##params = {"columns":",".join(FIELDS),
##          "filters":[{"paramName":"program","values":[program]}]}


#columns = ','.join(RET_COLUMNS)
#service = SERVICES['Caom_search']
#progID = program
#mjd_min = 59215
#start_date = mjd_min
#params = {"columns":columns,
#              "filters":[
#                  {"paramName":"obs_collection","values":["JWST"]},
#                  {"paramName":"instrument_name","values":[instrument]},
#                  {"paramName":"proposal_id","values":[progID]},
#                  {"paramName":"t_min",
#                    "values":[{"min":mjd_min,"max":mjd_min+60.}]},
#              ]}
#instr_obs = JwstObs.service_request(service, params)
# 
## Narrow the results to unique matches
#obs_table = table.unique(table.vstack(instr_obs))

In [126]:
#obsTable = observation_query(JwstObs, progID, instrument, start_date)

In [122]:
#print(obs_table)

proposal_id ...                  obs_id                
----------- ... ---------------------------------------
      00675 ... jw00675-c1002_t010_nirspec_g140h-f100lp
      00675 ... jw00675-c1002_t010_nirspec_g235h-f170lp
      00675 ... jw00675-c1002_t010_nirspec_g395h-f290lp
      00675 ... jw00675-c1002_t012_nirspec_g140h-f100lp
      00675 ... jw00675-c1002_t012_nirspec_g235h-f170lp
      00675 ... jw00675-c1002_t012_nirspec_g395h-f290lp
      00675 ...  jw00675-o006_t003_nirspec_g395h-f290lp
      00675 ...  jw00675-o021_t010_nirspec_g140h-f100lp
      00675 ...  jw00675-o021_t010_nirspec_g235h-f170lp
      00675 ...  jw00675-o021_t010_nirspec_g395h-f290lp
        ... ...                                     ...
      00675 ...          jw00675023001_03101_00002_nrs1
      00675 ...          jw00675023001_06101_00001_nrs1
      00675 ...          jw00675023001_06101_00001_nrs2
      00675 ...          jw00675023001_06101_00002_nrs1
      00675 ...          jw00675023001_06101_000

# Find Files, Download, and Open for each of the Data Modes

NIRSpec MOS\
NIRSpec IFU\
NIRSpec FS\
NIRSpec BOTS\
\
MIRI Imager\
MIRI LRS\
MIRI MRS\
MIRI Coronagraphic Imaging\
\
NIRCam Imaging\
NIRCam Coronagraphic Imaging\
NIRCam Grism TSO\
NIRCam WFSS\
\
NIRISS Imaging\
NIRISS WFSS\
NIRISS SOSS\
NIRISS AMI\

## NIRSpec MOS

No data in Rossy's table is green.

## NIRSpec IFU

In [621]:
instrument = 'Nirspec'
program = '00675'
start_date = 59215
#program = '00626'
#start_date = 59081
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
filt='F100LP'

extension='_s3d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55734929           JWST ... EXCLUSIVE_ACCESS           3
55734873           JWST ... EXCLUSIVE_ACCESS           3
55734548           JWST ... EXCLUSIVE_ACCESS           2
55734562           JWST ... EXCLUSIVE_ACCESS           2
55734553           JWST ... EXCLUSIVE_ACCESS           2
55734557           JWST ... EXCLUSIVE_ACCESS           2
55734565           JWST ... EXCLUSIVE_ACCESS           2
55734568           JWST ... EXCLUSIVE_ACCESS           2
55734121           JWST ... EXCLUSIVE_ACCESS           2
55734122           JWST ... EXCLUSIVE_ACCESS           2
55734410           JWST ... EXCLUSIVE_ACCESS           2
55734425           JWST ... EXCLUSIVE_ACCESS           2
55734428           JWST ... EXCLUSIVE_ACCESS           2
55734414           JWST ... EXCLUSIVE_ACCESS           2
55733823           JWST ... EXCLUSIVE_ACCESS           2
55733824           JWST ... EXC

In [622]:
#S3D and #X1D

script = fetch_files(progID, instrument, start_date, filt, 'lp_s3d.fits', True, True)
script = fetch_files(progID, instrument, start_date, filt, 'lp_x1d.fits', True, True)

jw00675-o021_t010_nirspec_g140h-f100lp_s3d.fits
jw00675-o022_t012_nirspec_g140h-f100lp_s3d.fits
jw00675-o021_t010_nirspec_g140h-f100lp_x1d.fits
jw00675-o022_t012_nirspec_g140h-f100lp_x1d.fits


In [591]:
filename = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/IFU/00626064/jw00626-o064_t007_nirspec_tg235h_g235h-f170lp_s3d.fits'

In [585]:
cubeviz = CubeViz()
cubeviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [593]:
cubeviz.load_data(filename)

2021-06-11 11:28:27,542 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-11 11:28:27,757 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-11 11:28:27,761 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-11 11:28:29,080 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-11 11:28:29,574 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-11 11:28:29,985 - stpipe - WARNING - No spectral axis found; header may be non-compliant.


In [594]:
from jdaviz import SpecViz
specviz = SpecViz()

In [595]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [596]:
spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/IFU/00626064/jw00626-o064_t007_nirspec_tg235h_g235h-f170lp_x1d.fits'
specviz.load_spectrum(spectrum, "IFU 1D")

RuntimeError: Keyword SRCTYPE is None.  It should be 'POINT' or 'EXTENDED'. Can't decide between `flux` and `surf_bright` columns.

## NIRSpec FS

In [223]:
instrument = 'Nirspec'
program = '00675'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
mjd_min = 59215
start_date = mjd_min
filt='CLEAR'



In [224]:
extension='_s2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55722833           JWST ... EXCLUSIVE_ACCESS           2
55722774           JWST ... EXCLUSIVE_ACCESS           2
55722641           JWST ... EXCLUSIVE_ACCESS           2
55722637           JWST ... EXCLUSIVE_ACCESS           2
55722610           JWST ... EXCLUSIVE_ACCESS           2
55722738           JWST ... EXCLUSIVE_ACCESS           2
55722672           JWST ... EXCLUSIVE_ACCESS           2
55722817           JWST ... EXCLUSIVE_ACCESS           2
55722686           JWST ... EXCLUSIVE_ACCESS           2
55722662           JWST ... EXCLUSIVE_ACCESS           2
55722644           JWST ... EXCLUSIVE_ACCESS           2
55722679           JWST ... EXCLUSIVE_ACCESS           2
55722811           JWST ... EXCLUSIVE_ACCESS           2
55722713           JWST ... EXCLUSIVE_ACCESS           2
55722667           JWST ... EXCLUSIVE_ACCESS           2
55722692           JWST ... EXC

In [225]:
script = fetch_files(progID, instrument, start_date, filt, '_s2d.fits', True, True)


jw00675011001_03102_00001_nrs1_s2d.fits
jw00675011001_03102_00002_nrs1_s2d.fits
jw00675011001_03102_00003_nrs1_s2d.fits
jw00675011001_03102_00004_nrs1_s2d.fits
jw00675011001_03102_00005_nrs1_s2d.fits
jw00675011001_03102_00006_nrs1_s2d.fits
jw00675011001_03102_00007_nrs1_s2d.fits
jw00675011001_03102_00008_nrs1_s2d.fits
jw00675011001_03102_00009_nrs1_s2d.fits
jw00675011001_03104_00001_nrs1_s2d.fits
jw00675011001_03104_00002_nrs1_s2d.fits
jw00675011001_03104_00003_nrs1_s2d.fits
jw00675011001_03104_00004_nrs1_s2d.fits
jw00675011001_03104_00005_nrs1_s2d.fits
jw00675011001_03104_00006_nrs1_s2d.fits
jw00675011001_03104_00007_nrs1_s2d.fits
jw00675011001_03104_00008_nrs1_s2d.fits
jw00675011001_03104_00009_nrs1_s2d.fits


In [227]:
script = fetch_files(progID, instrument, start_date, filt, '_x1d.fits', True, True)



jw00675011001_03102_00001_nrs1_x1d.fits
jw00675011001_03102_00002_nrs1_x1d.fits
jw00675011001_03102_00003_nrs1_x1d.fits
jw00675011001_03102_00004_nrs1_x1d.fits
jw00675011001_03102_00005_nrs1_x1d.fits
jw00675011001_03102_00006_nrs1_x1d.fits
jw00675011001_03102_00007_nrs1_x1d.fits
jw00675011001_03102_00008_nrs1_x1d.fits
jw00675011001_03102_00009_nrs1_x1d.fits
jw00675011001_03104_00001_nrs1_x1d.fits
jw00675011001_03104_00002_nrs1_x1d.fits
jw00675011001_03104_00003_nrs1_x1d.fits
jw00675011001_03104_00004_nrs1_x1d.fits
jw00675011001_03104_00005_nrs1_x1d.fits
jw00675011001_03104_00006_nrs1_x1d.fits
jw00675011001_03104_00007_nrs1_x1d.fits
jw00675011001_03104_00008_nrs1_x1d.fits
jw00675011001_03104_00009_nrs1_x1d.fits


In [623]:
from jdaviz import SpecViz
specviz = SpecViz()

In [624]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [626]:
#NOPS_spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/FS/jw00675011001_03102_00001_nrs1/jw00675011001_03102_00001_nrs1_x1d.fits'
brian_spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/FS/jw00636-o019_s00002_nirspec_f100lp-g140m-s200a2-subs200a2_x1d.fits'
specviz.load_spectrum(brian_spectrum, "FS 1D")

2021-06-11 13:01:34,274 - stpipe - WARNING - No data item with id '711997a4-178b-40ff-98b9-b5f850326883' found in viewer 'e4e11c08-94fa-4b60-a257-4fdaa8aa7669'.


In [627]:
from jdaviz import Specviz2d
specviz2d = Specviz2d()

In [628]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz2d.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [630]:
#NOPS_spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/FS/jw00675011001_03102_00001_nrs1/jw00675011001_03102_00001_nrs1_s2d.fits'
brian_spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/FS/jw00636-o019_s00002_nirspec_f100lp-g140m-s200a2-subs200a2_s2d.fits'

specviz2d.load_data(brian_spectrum)

## NIRSpec BOTS


In [256]:
instrument = 'Nirspec'
program = '00626'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
#mjd_min = 59215
mjd_min = 59061
start_date = mjd_min
filt='F170LP'


extension='_x1dints.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)  

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55727621           JWST ... EXCLUSIVE_ACCESS           2
55727386           JWST ... EXCLUSIVE_ACCESS           2
55727311           JWST ... EXCLUSIVE_ACCESS           2
55727264           JWST ... EXCLUSIVE_ACCESS           2


In [258]:
script = fetch_files(progID, instrument, start_date, filt, '_x1dints.fits', True, True)


jw00626062001_03101_00001-seg001_nrs1_x1dints.fits
jw00626062001_03101_00001-seg001_nrs2_x1dints.fits
jw00626062001_03101_00001-seg002_nrs1_x1dints.fits
jw00626062001_03101_00001-seg002_nrs2_x1dints.fits


## NIRISS Imaging


In [632]:
instrument = 'Niriss'
program = '00625'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
mjd_min = 58961
start_date = mjd_min
filt='CLEAR'


extension='_i2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 

 obsID   obs_collection ... dataRights calib_level
-------- -------------- ... ---------- -----------
55717358           JWST ...     PUBLIC           3
55717300           JWST ...     PUBLIC           3
55735200           JWST ...     PUBLIC           3
55735199           JWST ...     PUBLIC           3
55717325           JWST ...     PUBLIC           3
55735198           JWST ...     PUBLIC           3
55717323           JWST ...     PUBLIC           3
55717210           JWST ...     PUBLIC           3
55717283           JWST ...     PUBLIC           3
55717363           JWST ...     PUBLIC           3
     ...            ... ...        ...         ...
55728783           JWST ...     PUBLIC           2
55728835           JWST ...     PUBLIC           2
55728838           JWST ...     PUBLIC           2
55728806           JWST ...     PUBLIC           2
55728798           JWST ...     PUBLIC           2
55728799           JWST ...     PUBLIC           2
55728809           JWST ...    

In [633]:
script = fetch_files(progID, instrument, start_date, filt, '_i2d.fits', True, True)


jw00625-o039_t007_niriss_clear-f200w_i2d.fits
jw00625-o054_t007_niriss_clear-f090w_i2d.fits
jw00625-o054_t007_niriss_clear-f115w_i2d.fits
jw00625-o054_t007_niriss_clear-f150w_i2d.fits
jw00625-o054_t007_niriss_clear-f200w_i2d.fits
jw00625-o070_t001_niriss_clear-f090w_i2d.fits
jw00625-o070_t001_niriss_clear-f150w_i2d.fits
jw00625-o072_t006_niriss_clear-f090w_i2d.fits
jw00625-o072_t006_niriss_clear-f150w_i2d.fits
jw00625-o072_t006_niriss_clear-f200w_i2d.fits
jw00625-o075_t007_niriss_clear-f200w_i2d.fits
       jw00625039001_02101_00001_nis_i2d.fits
       jw00625039002_02101_00001_nis_i2d.fits
       jw00625040001_02101_00001_nis_i2d.fits
       jw00625040001_02103_00001_nis_i2d.fits
       jw00625040001_02103_00002_nis_i2d.fits
       jw00625040001_02105_00001_nis_i2d.fits
       jw00625040002_02101_00001_nis_i2d.fits
       jw00625040002_02103_00001_nis_i2d.fits
       jw00625040002_02103_00002_nis_i2d.fits
       jw00625040002_02105_00001_nis_i2d.fits
       jw00625054001_02101_00001_n

In [272]:
niriss_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRISS/Imaging/jw00625-o072_t006_niriss_clear-f090w/jw00625-o072_t006_niriss_clear-f090w_i2d.fits'

In [273]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [274]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nirirss_imaging, data_label='NIRISS Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## NIRISS WFSS

In [634]:
instrument = 'Niriss'
program = '00625'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
#mjd_min = 58961

start_date = 59000
filt='CLEAR'

extension='_x1d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
#products = product_query(JwstObs, obsTable)
#mask = product_filter(products, [extension], True)
#selected_products = products[mask]
#print(selected_products) 
print(obsTable)

proposal_id ...                obs_id               
----------- ... ------------------------------------
      00625 ...        jw00625040002_02101_00001_nis
      00625 ...        jw00625047001_02101_00007_nis
      00625 ... jw00625-o072_t006_niriss_clear-f090w
      00625 ...        jw00625047001_02101_00006_nis
      00625 ... jw00625-o070_t001_niriss_clear-f090w
      00625 ...        jw00625047001_02101_00002_nis
      00625 ...        jw00625047001_02101_00011_nis
      00625 ...        jw00625071001_03101_00001_nis
      00625 ... jw00625-o054_t007_niriss_clear-f200w
      00625 ...        jw00625040001_02103_00001_nis
        ... ...                                  ...
      00625 ... jw00625-o039_t007_niriss_clear-f200w
      00625 ...        jw00625040002_02103_00002_nis
      00625 ...        jw00625047001_02101_00008_nis
      00625 ... jw00625-o054_t007_niriss_clear-f115w
      00625 ...        jw00625048001_02101_00001_nis
      00625 ...        jw00625047001_02101_000

In [315]:
script = fetch_files(progID, instrument, start_date, filt, '_x1d.fits', True, True)


KeyboardInterrupt: 

In [ ]:
nircam_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRISS/Imaging/jw00625-o072_t006_niriss_clear-f090w/jw00625-o072_t006_niriss_clear-f090w_i2d.fits'

## NIRISS SOSS

In [321]:
instrument = 'Niriss'
program = '00721'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
#mjd_min = 58961

start_date = 59030
#start_data = 59215
filt='CLEAR'

extension='_x1dints.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55772989           JWST ... EXCLUSIVE_ACCESS           3
55772924           JWST ... EXCLUSIVE_ACCESS           2
55772922           JWST ... EXCLUSIVE_ACCESS           2
55772923           JWST ... EXCLUSIVE_ACCESS           2


In [322]:
script = fetch_files(progID, instrument, start_date, filt, '_x1dints.fits', True, True)



jw00721-o033_t008_niriss_clear-gr700xd-substrip96_x1dints.fits
             jw00721033001_03101_00001-seg001_nis_x1dints.fits
             jw00721033001_03101_00001-seg002_nis_x1dints.fits
             jw00721033001_03101_00001-seg003_nis_x1dints.fits


## NIRISS AMI - Failing

In [324]:
instrument = 'Niriss'
program = '00625'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program

start_data = 59215
filt='F380M'

extension='_amiavg.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
#print(obsTable)



 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55721117           JWST ... EXCLUSIVE_ACCESS           3


## NIRCam Imager - Working

In [8]:
nircam_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRCam/Imaging/images_level3/LMC_F070W_nrcb1_combined_PrimaryDither_1_SubpxDither_4_i2d.fits'

  and should_run_async(code)

2021-06-03 21:33:30,190 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [9]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-03 21:33:30,980 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



In [10]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nircam_imaging, data_label='NIRCam Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

2021-06-03 21:33:31,892 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jwst/datamodels/drizproduct.py:10: DeprecationWarning: DrizProduct is deprecated and will be removed.  Use ImageModel.
  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-03 21:33:32,369 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy sc

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## NIRCam Imager

In [561]:
instrument = 'Nircam'
program = '00720'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program

start_date = 58520
filt='F090W'

extension='_i2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55744142           JWST ... EXCLUSIVE_ACCESS           3
55744144           JWST ... EXCLUSIVE_ACCESS           3
55744146           JWST ... EXCLUSIVE_ACCESS           3
55717720           JWST ... EXCLUSIVE_ACCESS           2
55717698           JWST ... EXCLUSIVE_ACCESS           2
55717736           JWST ... EXCLUSIVE_ACCESS           2
55717827           JWST ... EXCLUSIVE_ACCESS           2
55717794           JWST ... EXCLUSIVE_ACCESS           2
55717434           JWST ... EXCLUSIVE_ACCESS           2
55717418           JWST ... EXCLUSIVE_ACCESS           2
     ...            ... ...              ...         ...
55717768           JWST ... EXCLUSIVE_ACCESS           2
55717866           JWST ... EXCLUSIVE_ACCESS           2
55717514           JWST ... EXCLUSIVE_ACCESS           2
55717529           JWST ... EXCLUSIVE_ACCESS           2
55717496           JWST ... EXC

In [562]:
script = fetch_files(progID, instrument, start_date, filt, '_i2d.fits', True, True)

    jw00720-o009_t002_nircam_f090w_i2d.fits
   jw00720-o009_t002_nircam_f322w2_i2d.fits
    jw00720-o010_t002_nircam_f277w_i2d.fits
   jw00720009001_03101_00001_nrcb1_i2d.fits
   jw00720009001_03101_00001_nrcb2_i2d.fits
   jw00720009001_03101_00001_nrcb3_i2d.fits
   jw00720009001_03101_00001_nrcb4_i2d.fits
jw00720009001_03101_00001_nrcblong_i2d.fits
   jw00720009001_03101_00002_nrcb1_i2d.fits
   jw00720009001_03101_00002_nrcb2_i2d.fits
   jw00720009001_03101_00002_nrcb3_i2d.fits
   jw00720009001_03101_00002_nrcb4_i2d.fits
jw00720009001_03101_00002_nrcblong_i2d.fits
   jw00720009002_03101_00001_nrcb1_i2d.fits
   jw00720009002_03101_00001_nrcb2_i2d.fits
   jw00720009002_03101_00001_nrcb3_i2d.fits
   jw00720009002_03101_00001_nrcb4_i2d.fits
jw00720009002_03101_00001_nrcblong_i2d.fits
   jw00720009002_03101_00002_nrcb1_i2d.fits
   jw00720009002_03101_00002_nrcb2_i2d.fits
   jw00720009002_03101_00002_nrcb3_i2d.fits
   jw00720009002_03101_00002_nrcb4_i2d.fits
jw00720009002_03101_00002_nrcblo

In [ ]:
nircam_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRCAM/Coronagraphic_Imaging/jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r/jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits'

In [ ]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [ ]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nircam_imaging, data_label='NIRCam Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

## NIRCam Coronagraphic Imaging


In [533]:
instrument = 'Nircam'
program = '00628'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program

start_data = 59266
filt='F335M'

extension='_i2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55729216           JWST ... EXCLUSIVE_ACCESS           3
55729216           JWST ... EXCLUSIVE_ACCESS           3
55729216           JWST ... EXCLUSIVE_ACCESS           3
55729216           JWST ... EXCLUSIVE_ACCESS           3
55729227           JWST ... EXCLUSIVE_ACCESS           3
55729227           JWST ... EXCLUSIVE_ACCESS           3
55729227           JWST ... EXCLUSIVE_ACCESS           3
55729227           JWST ... EXCLUSIVE_ACCESS           3
55729201           JWST ... EXCLUSIVE_ACCESS           3
55729201           JWST ... EXCLUSIVE_ACCESS           3
55729201           JWST ... EXCLUSIVE_ACCESS           3
55729201           JWST ... EXCLUSIVE_ACCESS           3
55729209           JWST ... EXCLUSIVE_ACCESS           3
55729209           JWST ... EXCLUSIVE_ACCESS           3
55729209           JWST ... EXCLUSIVE_ACCESS           3
55729209           JWST ... EXC

In [534]:
script = fetch_files(progID, instrument, start_date, filt, '_i2d.fits', True, True)



jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1024_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1024_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1024_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1024_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1029_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1029_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1029_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1029_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1039_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1039_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1039_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits
jw00628-c1039_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits


In [514]:
nircam_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRCAM/Coronagraphic_Imaging/jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r/jw00628-c1007_t016_nircam_f250m-maskrnd-sub320a335r_i2d.fits'


In [515]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [516]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nircam_imaging, data_label='NIRCam Coronagraphic Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

2021-06-11 07:25:17,148 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jwst/datamodels/drizproduct.py:10: DeprecationWarning: DrizProduct is deprecated and will be removed.  Use ImageModel.
  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-11 07:25:17,613 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy sc

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## NIRCam Grism TSO

In [492]:
instrument = 'Nircam'
program = '00721'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program

start_data = 59235
filt='F444W'

extension='_x1dints.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55729179           JWST ... EXCLUSIVE_ACCESS           3
55726820           JWST ... EXCLUSIVE_ACCESS           2


In [493]:
script = fetch_files(progID, instrument, start_date, filt, 'x1dints.fits', True, True)


jw00721-o012_t004_nircam_f444w-grismr-subgrism256_x1dints.fits
        jw00721012001_03103_00001-seg001_nrcalong_x1dints.fits


In [494]:
nircam_bots = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRCAM/Grism_TSO/jw00721-o012_t004_nircam_f444w-grismr-subgrism256/jw00721-o012_t004_nircam_f444w-grismr-subgrism256_x1dints.fits'


## NIRCam WFSS

In [ ]:
instrument = 'Nircam'
program = '00624'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program

start_data = 59215
filt='F380M'

extension='_amiavg.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products) 
#print(obsTable)

## MIRI Imaging

In [339]:
instrument = 'MIRI'
program = '00623'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
start_date = 58803
filt='F1500W'

extension='sub128_i2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
#print(obsTable)

 obsID   obs_collection ... dataRights calib_level
-------- -------------- ... ---------- -----------
55745230           JWST ...     PUBLIC           3
55745219           JWST ...     PUBLIC           3


In [340]:
script = fetch_files(progID, instrument, start_date, filt, 'sub128_i2d.fits', True, True)



jw00623-o046_t007_miri_f1500w-sub128_i2d.fits
jw00623-o060_t009_miri_f1500w-sub128_i2d.fits


In [341]:
miri_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/Imaging/jw00623-o046_t007_miri_f1500w-sub128/jw00623-o046_t007_miri_f1500w-sub128_i2d.fits'

In [344]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [345]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(miri_imaging, data_label='MIRI Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## MIRI LRS Slit - 1D, and 2D Currently failing

In [377]:
instrument = 'Miri'
program = '00623'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
start_date = 59000
filt='P750L'

extension='_s2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55745234           JWST ...           PUBLIC           3
55724991           JWST ... EXCLUSIVE_ACCESS           2
55725088           JWST ... EXCLUSIVE_ACCESS           2
55725172           JWST ... EXCLUSIVE_ACCESS           2
55725200           JWST ... EXCLUSIVE_ACCESS           2
55724886           JWST ... EXCLUSIVE_ACCESS           2
55725018           JWST ... EXCLUSIVE_ACCESS           2
55725119           JWST ... EXCLUSIVE_ACCESS           2
55724925           JWST ... EXCLUSIVE_ACCESS           2
55725211           JWST ... EXCLUSIVE_ACCESS           2
55725161           JWST ... EXCLUSIVE_ACCESS           2
55724946           JWST ... EXCLUSIVE_ACCESS           2
55725064           JWST ... EXCLUSIVE_ACCESS           2
55725494           JWST ... EXCLUSIVE_ACCESS           2
55725646           JWST ... EXCLUSIVE_ACCESS           2
55725430           JWST ... EXC

In [378]:
script = fetch_files(progID, instrument, start_date, filt, '_s2d.fits', True, True)



      jw00623-o074_t015_miri_clear_s2d.fits
jw00623074001_02101_00001_mirimage_s2d.fits
jw00623074001_02101_00002_mirimage_s2d.fits
jw00623074001_02103_00001_mirimage_s2d.fits
jw00623074001_02103_00002_mirimage_s2d.fits
jw00623080001_04102_00001_mirimage_s2d.fits
jw00623080001_04102_00002_mirimage_s2d.fits
jw00623080001_04104_00001_mirimage_s2d.fits
jw00623080001_04104_00002_mirimage_s2d.fits
jw00623081001_03101_00001_mirimage_s2d.fits
jw00623081001_03101_00002_mirimage_s2d.fits
jw00623081001_03103_00001_mirimage_s2d.fits
jw00623081001_03103_00002_mirimage_s2d.fits
jw00623083001_03103_00001_mirimage_s2d.fits
jw00623083001_03103_00002_mirimage_s2d.fits
jw00623083001_03106_00001_mirimage_s2d.fits
jw00623083001_03106_00002_mirimage_s2d.fits
jw00623084001_02101_00001_mirimage_s2d.fits
jw00623084001_02101_00002_mirimage_s2d.fits
jw00623084001_02103_00001_mirimage_s2d.fits
jw00623084001_02103_00002_mirimage_s2d.fits
jw00623085001_03103_00001_mirimage_s2d.fits
jw00623085001_03103_00002_mirima

In [379]:
script = fetch_files(progID, instrument, start_date, filt, '_x1d.fits', True, True)




      jw00623-o074_t015_miri_clear_x1d.fits
jw00623074001_02101_00001_mirimage_x1d.fits
jw00623074001_02101_00002_mirimage_x1d.fits
jw00623074001_02103_00001_mirimage_x1d.fits
jw00623074001_02103_00002_mirimage_x1d.fits
jw00623080001_04102_00001_mirimage_x1d.fits
jw00623080001_04102_00002_mirimage_x1d.fits
jw00623080001_04104_00001_mirimage_x1d.fits
jw00623080001_04104_00002_mirimage_x1d.fits
jw00623081001_03101_00001_mirimage_x1d.fits
jw00623081001_03101_00002_mirimage_x1d.fits
jw00623081001_03103_00001_mirimage_x1d.fits
jw00623081001_03103_00002_mirimage_x1d.fits
jw00623082001_04103_00001_mirimage_x1d.fits
jw00623082001_04103_00002_mirimage_x1d.fits
jw00623082001_04103_00003_mirimage_x1d.fits
jw00623082001_04103_00004_mirimage_x1d.fits
jw00623082001_04103_00005_mirimage_x1d.fits
jw00623082001_04103_00006_mirimage_x1d.fits
jw00623082001_04103_00007_mirimage_x1d.fits
jw00623082001_04103_00008_mirimage_x1d.fits
jw00623082001_04103_00009_mirimage_x1d.fits
jw00623082001_04103_00010_mirima

In [386]:
from jdaviz import Specviz2d
specviz2d = Specviz2d()

In [387]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz2d.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [388]:
spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/LRS/jw00623-o074_t015_miri_clear/jw00623-o074_t015_miri_clear_s2d.fits'
specviz2d.load_data(spectrum)

KeyError: "Keyword 'WAVEND' not found."

In [389]:
from jdaviz import SpecViz
specviz = SpecViz()

In [390]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [391]:
spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/LRS/jw00623-o074_t015_miri_clear/jw00623-o074_t015_miri_clear_x1d.fits'
specviz.load_spectrum(spectrum, "LRS 1D")

RuntimeError: Keyword SRCTYPE is None.  It should be 'POINT' or 'EXTENDED'. Can't decide between `flux` and `surf_bright` columns.

## MIRI LRS Slitless - X1DINTS Currently failing

In [609]:
instrument = 'Miri'
program = '00623'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
start_date = 59009
filt='P750L'

extension='p750l-slitlessprism_x1dints.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
#print(obsTable)

obsID obs_collection obs_id ... parent_obsid dataRights calib_level
----- -------------- ------ ... ------------ ---------- -----------


In [433]:
script = fetch_files(progID, instrument, start_date, filt, '_x1dints.fits', True, True)


jw00623-o027_t001_miri_p750l-slitlessprism_x1dints.fits
jw00623-o027_t001_miri_p750l-slitlessprism_x1dints.fits
     jw00623-o027_t001_miri_p750l-subprism_x1dints.fits
     jw00623-o027_t001_miri_p750l-subprism_x1dints.fits
jw00623-o040_t009_miri_p750l-slitlessprism_x1dints.fits
 jw00623027001_03103_00001-seg001_mirimage_x1dints.fits
 jw00623027001_03103_00001-seg001_mirimage_x1dints.fits
 jw00623040001_02101_00001-seg001_mirimage_x1dints.fits
 jw00623055001_04103_00001-seg001_mirimage_x1dints.fits


## MIRI MRS

In [613]:
instrument = 'Miri'
program = '00623'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
start_date = 59139
filt=''

extension='_s3d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
print(obsTable)

obsID obs_collection obs_id ... parent_obsid dataRights calib_level
----- -------------- ------ ... ------------ ---------- -----------
proposal_id dataURL obsid obs_id
----------- ------- ----- ------


In [566]:
script = fetch_files(progID, instrument, start_date, filt, '_s3d.fits', True, True)


 jw00623087001_05101_00001_mirifulong_s3d.fits
jw00623087001_05101_00001_mirifushort_s3d.fits
 jw00623087001_05101_00002_mirifulong_s3d.fits
jw00623087001_05101_00002_mirifushort_s3d.fits
 jw00623087001_07101_00001_mirifulong_s3d.fits
jw00623087001_07101_00001_mirifushort_s3d.fits
 jw00623087001_07101_00002_mirifulong_s3d.fits
jw00623087001_07101_00002_mirifushort_s3d.fits
 jw00623087001_09101_00001_mirifulong_s3d.fits
jw00623087001_09101_00001_mirifushort_s3d.fits
 jw00623087001_09101_00002_mirifulong_s3d.fits
jw00623087001_09101_00002_mirifushort_s3d.fits
 jw00623087001_11101_00001_mirifulong_s3d.fits
jw00623087001_11101_00001_mirifushort_s3d.fits
 jw00623087001_11101_00002_mirifulong_s3d.fits
jw00623087001_11101_00002_mirifushort_s3d.fits
 jw00623087001_13101_00001_mirifulong_s3d.fits
jw00623087001_13101_00001_mirifushort_s3d.fits
 jw00623087001_13101_00002_mirifulong_s3d.fits
jw00623087001_13101_00002_mirifushort_s3d.fits
 jw00623087001_15101_00001_mirifulong_s3d.fits
jw00623087001

In [474]:
script = fetch_files(progID, instrument, start_date, filt, '_x1d.fits', True, True)


   jw00623083001_03103_00001_mirimage_x1d.fits
   jw00623083001_03103_00002_mirimage_x1d.fits
   jw00623083001_03106_00001_mirimage_x1d.fits
   jw00623083001_03106_00002_mirimage_x1d.fits
   jw00623084001_02101_00001_mirimage_x1d.fits
   jw00623084001_02101_00002_mirimage_x1d.fits
   jw00623084001_02103_00001_mirimage_x1d.fits
   jw00623084001_02103_00002_mirimage_x1d.fits
   jw00623085001_03103_00001_mirimage_x1d.fits
   jw00623085001_03103_00002_mirimage_x1d.fits
   jw00623086001_03103_00001_mirimage_x1d.fits
   jw00623086001_03103_00002_mirimage_x1d.fits
   jw00623086001_03103_00003_mirimage_x1d.fits
   jw00623086001_03103_00004_mirimage_x1d.fits
   jw00623086001_03103_00005_mirimage_x1d.fits
   jw00623086001_03103_00006_mirimage_x1d.fits
   jw00623086001_03103_00007_mirimage_x1d.fits
   jw00623086001_03103_00008_mirimage_x1d.fits
   jw00623086001_03103_00009_mirimage_x1d.fits
   jw00623086001_03103_00010_mirimage_x1d.fits
   jw00623086001_03103_00011_mirimage_x1d.fits
   jw00623086

In [477]:
from jdaviz.app import Application

app = Application(configuration='cubeviz')# when doing Cubeviz, first run this block.
app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [479]:
ch1short_cubefile='/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/MRS/jw00623-o087_t017_miri_ch1-short_s3d.fits'
#ch1short_cubefile='combine_dithers_all_exposures_ch1-2-3-4-shortlongmedium-_s3d.fits'
app.load_data(ch1short_cubefile)# When this block is run, the spectrum will appear in the Cubeviz viewer above.

2021-06-10 22:00:21,098 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-10 22:00:21,137 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-10 22:00:21,141 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-10 22:00:21,732 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-10 22:00:21,955 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-10 22:00:22,215 - stpipe - WARNING - No spectral axis found; header may be non-compliant.


In [480]:
from jdaviz import SpecViz
specviz = SpecViz()
#Developer note: How do you scroll interactingvely with the mouse?
#specviz.app

In [481]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [482]:
spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/MRS/jw00623-o087_t017_miri_ch1-short_x1d.fits'
specviz.load_spectrum(spectrum, "MRS 1D")

## MIRI Coronagraphic Imaging - 4QPM

In [485]:
instrument = 'MIRI'
program = '00628'
columns = ','.join(RET_COLUMNS)
service = SERVICES['Caom_search']
progID = program
start_date = 58627
filt='F1065C'

extension='_i2d.fits'
JwstObs = set_server_endpoints(SERVER)
obsTable = observation_query(JwstObs, progID, instrument, start_date, filt)
products = product_query(JwstObs, obsTable)
mask = product_filter(products, [extension], True)
selected_products = products[mask]
print(selected_products)
#print(obsTable)

 obsID   obs_collection ...    dataRights    calib_level
-------- -------------- ... ---------------- -----------
55729222           JWST ... EXCLUSIVE_ACCESS           3
55729222           JWST ... EXCLUSIVE_ACCESS           3
55729222           JWST ... EXCLUSIVE_ACCESS           3
55729222           JWST ... EXCLUSIVE_ACCESS           3
55729205           JWST ... EXCLUSIVE_ACCESS           3
55729205           JWST ... EXCLUSIVE_ACCESS           3
55729205           JWST ... EXCLUSIVE_ACCESS           3
55729205           JWST ... EXCLUSIVE_ACCESS           3
55729225           JWST ... EXCLUSIVE_ACCESS           3
55729225           JWST ... EXCLUSIVE_ACCESS           3
55729197           JWST ... EXCLUSIVE_ACCESS           3
55729197           JWST ... EXCLUSIVE_ACCESS           3
55729197           JWST ... EXCLUSIVE_ACCESS           3
55729197           JWST ... EXCLUSIVE_ACCESS           3
55729187           JWST ... EXCLUSIVE_ACCESS           3
55729187           JWST ... EXC

In [486]:
script = fetch_files(progID, instrument, start_date, filt, '_i2d.fits', True, True)



jw00628-c1005_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1005_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1005_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1005_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1022_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1022_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1022_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1022_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1025_t016_miri_f1065c-mask1065_i2d.fits
jw00628-c1025_t016_miri_f1065c-mask1065_i2d.fits
jw00628-c1027_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1027_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1027_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1027_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1037_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1037_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1037_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1037_t010_miri_f2300c-masklyot_i2d.fits
jw00628-c1040_t016_miri_f1065c-mask1065_i2d.fits
jw00628-c1040_t016_miri_f1065c-mask1065_i2d.fits


In [487]:
miri_imaging = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/CORONAGRAPHIC_IMAGING/jw00628-c1025_t016_miri_f1065c-mask1065/jw00628-c1025_t016_miri_f1065c-mask1065_i2d.fits'

In [488]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [489]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(miri_imaging, data_label='MIRI Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …